# Функции transform_inn_col(), check_inn()

### Импорт модулей

In [1]:
from pyspark.sql import SparkSession

### Импорт библиотеки pysp_lib

In [2]:
from pysp_lib.check import *

### Инициализация Spark

In [3]:
spark = SparkSession.builder.appName("Test").master("local[*]").getOrCreate()

### Создаем тестовый dataframe

In [4]:
df = spark.createDataFrame([('OOO Руслан', '4707005850')
                               , ('ООО Агроторг', '7825706086')
                               , ('ООО Ригла', None)
                               , ('ПАО Сургутнефтегаз', '7724211288')
                               , ('ООО Альбион-2002', '8602060555')
                               , ('ПАО Щит', '5257056036')
                               , ('АО Эр-Телеком Холдинг', '')
                               , ('OOO Ромашка', None)
                               , ('АО Магистраль', None)
                               , ('ИП Макеев Сергей Николаевич', '131900231406')
                               , ('ИП Савкин Николай Степанович', '131400060294')
                               , ('ИП Анчина Валентина Николаевна', '132300043547')
                               , ('ИП Михайлова Нина Викторовна', '131400027307')
                               , ('ИП Веремеенко Сергей Евгеньевич', '132200001744')
                               , ('ИП Журавлев Анатолий Николаевич', '123456790A00')
                               , ('ИП Макаров Петр Владимирович', '000000000000')
                               , ('ООО Пальма', '0000000000')
                               , ('АО Завод', 'ASDFTV')
                               , ('ПАО Кот', '0001234567')
                               , ('ООО Веста', '0105020396')
                               , ('ООО Дом', '105020396')
                               , ('ЗАО Марс', None)
                               , ('ООО Пират', '4707005850')
                               , ('АО Музыка', '7825706086')
                               , ('ПАО Пластинка', '105047905')
                               , ('ООО Луна', '0101010083')
                               , ('АО Сон', None)]
                            , ['company', 'inn'])

Общее количество записей

In [5]:
df.count()

27

Количество нулевых значений

In [6]:
df.filter(f.col('inn').isNull()).count()

5

### Вызов функции transform_inn_col()

In [7]:
test_transform_func = transform_inn_col(df, 'inn')

Количество значений (нулевые значения отфильтрованы)

In [8]:
test_transform_func.count()

22

Смотрим изменившиеся колонки

In [9]:
test_transform_func.select('inn_fixed', 'company')\
                    .exceptAll(test_transform_func.select('inn', 'company'))\
                    .show()

+----------+--------------------+
| inn_fixed|             company|
+----------+--------------------+
|0105020396|             ООО Дом|
|          |            АО Завод|
|0105047905|       ПАО Пластинка|
|0123456790|ИП Журавлев Анато...|
+----------+--------------------+



### Вызов функции check_inn()

В качестве параметра передается колонка с ИНН предварительно не отредактированная

In [10]:
test_check_inn_func_non_fixed_inn = check_inn(test_transform_func, 'inn')

Считаем количество валидных ИНН

In [11]:
test_check_inn_func_non_fixed_inn.groupBy('inn_valid_flag').count().show()

+--------------+-----+
|inn_valid_flag|count|
+--------------+-----+
|             1|   14|
|             0|    8|
+--------------+-----+



В качестве параметра передается колонка с ИНН предварительно отредактированная

In [12]:
test_check_inn_func_fixed_inn = check_inn(test_transform_func, 'inn_fixed')

Считаем количество валидных ИНН (благодаря предварительному редактированию удалось увеличить количество валидных ИНН)

In [13]:
test_check_inn_func_fixed_inn.groupBy('inn_fixed_valid_flag').count().show()

+--------------------+-----+
|inn_fixed_valid_flag|count|
+--------------------+-----+
|                   1|   16|
|                   0|    6|
+--------------------+-----+



In [14]:
spark.stop()